In [1]:
import pandas as pd
import numpy as np
import urllib.request
import matplotlib.pyplot as plt
import time
from datetime import datetime, timezone

In [2]:
ts = int("1619063100")
print(datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))

2021-04-22 09:15:00


In [3]:
ts = int("1619098340")
print(datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))

2021-04-22 19:02:20


In [4]:
from collections import deque
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization, Bidirectional
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint, ModelCheckpoint
from sklearn import preprocessing

In [5]:
SEQ_LEN = 60  # how long of a preceeding sequence to collect for RNN
FUTURE_PERIOD_PREDICT = 3  # how far into the future are we trying to predict?
RATIO_TO_PREDICT = "c"
EPOCHS = 20  # how many passes through our data
BATCH_SIZE = 64  # how many batches? Try smaller batch if you're getting OOM (out of memory) errors.
NAME = f"{SEQ_LEN}-SEQ-{FUTURE_PERIOD_PREDICT}-PRED-{int(time.time())}"

In [6]:
def classify(current, future):
    if float(future) > float(current):  # if the future price is higher than the current, that's a buy, or a 1
        return 1
    else:  # otherwise... it's a 0!
        return 0

In [7]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

In [8]:
def preprocess_df(df):
    for col in df.columns:  # go through all of the columns
        df[col] = df[col].pct_change()  # pct change "normalizes" the different currencies (each crypto coin has vastly diff values, we're really more interested in the other coin's movements)
        df=clean_dataset(df) # remove the nas created by pct_change
#         df.dropna(inplace=True)
        df[col] = preprocessing.scale(df[col].values)  # scale between 0 and 1.

    df.dropna(inplace=True)  # cleanup again... jic.
    return df  # return X and y...and make X a numpy array!

# RealTime

In [9]:
model=tf.keras.models.load_model("models/FINAL_20EPOCH_MODEL/")

In [10]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_14 (LSTM)               (None, 60, 32)            4608      
_________________________________________________________________
dropout_21 (Dropout)         (None, 60, 32)            0         
_________________________________________________________________
batch_normalization_14 (Batc (None, 60, 32)            128       
_________________________________________________________________
lstm_15 (LSTM)               (None, 64)                24832     
_________________________________________________________________
dropout_22 (Dropout)         (None, 64)                0         
_________________________________________________________________
batch_normalization_15 (Batc (None, 64)                256       
_________________________________________________________________
dense_14 (Dense)             (None, 22)               

In [11]:
model.input_shape

(None, 60, 3)

In [21]:
dt = datetime(time.localtime().tm_year, time.localtime().tm_mon, time.localtime().tm_mday, 9, 15)
timestart = int(dt.timestamp())

In [22]:
starter=timestart
ender=timestart+(75*60)
# starter=int(time.time()-3600)
# buffPd=pd.DataFrame(columns=['t', 'c', 'o', 'h', 'l', 'v', 'vo', 's'])
Resolution=1
main_df=pd.DataFrame()
while(1):
    url = "https://tvc4.forexpros.com/9e947eed212d70ba03e313a75eb9a1f5/1616696634/56/56/23/history?symbol=17940&resolution="+str(Resolution)+"&from="+str(starter)+"&to="+str(ender)
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'

    headers={'User-Agent':user_agent,} 

    request=urllib.request.Request(url,None,headers)
    response = urllib.request.urlopen(request)

    data = response.read()
#     print(data)
#     print(len(pd.read_json(data)))
#     print(data)
    try:
        main_df=pd.read_json(data)
#         time.sleep(60)
    except:
        print("No data available yet!")
#         time.sleep(60) #This
    
    if len(main_df)>=75:
#         main_df=main_df[(main_df.index < 63)]
        
        url = "https://tvc4.forexpros.com/9e947eed212d70ba03e313a75eb9a1f5/1616696634/56/56/23/history?symbol=17940&resolution="+str(5)+"&from="+str(starter)+"&to="+str(int(time.time()))
        user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'

        headers={'User-Agent':user_agent,} 

        request=urllib.request.Request(url,None,headers)
        response = urllib.request.urlopen(request)

        data = response.read()
        main_dfV=pd.read_json(data)
        
        main_dfV=main_dfV[(main_dfV.index < 16)]
        
        npDf=np.array(main_df)
        npDfV=np.array(main_dfV)
        
        opti=0
        for i in npDfV[1:,:]:
            buff=i[0]

            for j in range(opti,npDf.shape[0]):
                if npDf[j][0]==buff:
                    opti=j
                    break

            npDf[j-4:j+1,-3]=i[-3]/5
        
        main_df=pd.DataFrame(npDf,columns=['t','c','o','h','l','v','vo','s'])
        
        prevPrice=str(np.array(main_df['c'])[-1])
        prevTime=int(np.array(main_df['t'])[-1])
        
        main_df['diff']=main_df.h-main_df.l
        main_df=main_df.drop(labels=['t','o','h','l','s','vo'],axis=1)

        validation_x = preprocess_df(main_df)
        print("Predicting!")
        Result=np.argmax(model.predict(np.array(validation_x)[-60:,:].reshape(1,60,3)))
        
#         dt=datetime.fromtimestamp(time.time()+300).strftime('%Y-%m-%d %H:%M:%S') #This
        
        dt=datetime.fromtimestamp(prevTime+60).strftime('%Y-%m-%d %H:%M:%S')
        
        if Result==0:
            print("The price of Nifty-50 will decrease! from " +prevPrice+" at "+dt)
        else:
            print("The price of Nifty-50 will increase! from " +prevPrice+" at "+dt)
#         time.sleep(200) #This
        
        starter+=63
        ender+=63
        
        main_df=pd.DataFrame()
    else:
#         time.sleep(60) #This
        pass

Predicting!
The price of Nifty-50 will decrease! from 14696.75 at 2021-04-06 10:31:00
Predicting!
The price of Nifty-50 will decrease! from 14706.650390625 at 2021-04-06 10:32:00
Predicting!
The price of Nifty-50 will decrease! from 14703.849609375 at 2021-04-06 10:33:00
Predicting!
The price of Nifty-50 will decrease! from 14708.0 at 2021-04-06 10:34:00
Predicting!
The price of Nifty-50 will decrease! from 14703.75 at 2021-04-06 10:35:00
Predicting!
The price of Nifty-50 will decrease! from 14707.599609375 at 2021-04-06 10:36:00
Predicting!
The price of Nifty-50 will decrease! from 14714.849609375 at 2021-04-06 10:37:00
Predicting!
The price of Nifty-50 will decrease! from 14713.0 at 2021-04-06 10:38:00
Predicting!
The price of Nifty-50 will decrease! from 14715.900390625 at 2021-04-06 10:39:00
Predicting!
The price of Nifty-50 will decrease! from 14712.849609375 at 2021-04-06 10:40:00
Predicting!
The price of Nifty-50 will increase! from 14724.25 at 2021-04-06 10:41:00
Predicting!
Th

KeyboardInterrupt: 

# Below code was for testing whole day's data at one go

In [38]:
# starter=timestart
starter=int(time.time()-3600)
# buffPd=pd.DataFrame(columns=['t', 'c', 'o', 'h', 'l', 'v', 'vo', 's'])
Resolution=5
main_df=pd.DataFrame()
while(1):
    url = "https://tvc4.forexpros.com/9e947eed212d70ba03e313a75eb9a1f5/1616696634/56/56/23/history?symbol=17940&resolution="+str(Resolution)+"&from="+str(starter)+"&to="+str(int(time.time()))
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'

    headers={'User-Agent':user_agent,} 

    request=urllib.request.Request(url,None,headers)
    response = urllib.request.urlopen(request)

    data = response.read()
#     print(data)
#     print(len(pd.read_json(data)))
#     print(data)
    try:
        main_df=pd.read_json(data)
#         time.sleep(60)
    except:
        print("No data available yet!")
        time.sleep(60)
    
    if len(main_df)>=13:
        main_df=main_df[(main_df.index < 13)]
        prevPrice=str(np.array(main_df['c'])[-1])
        main_df['diff']=main_df.h-main_df.l
        main_df=main_df.drop(labels=['t','o','h','l','s','vo'],axis=1)
        
        validation_x = preprocess_df(main_df)
        Result=np.argmax(model.predict(np.array(validation_x).reshape(1,12,3)))
        
        dt=datetime.fromtimestamp(time.time()+300).strftime('%Y-%m-%d %H:%M:%S')
        
        if Result==0:
            print("The price of Nifty-50 will decrease! from " +prevPrice+" at "+dt)
        else:
            print("The price of Nifty-50 will increase! from " +prevPrice+" at "+dt)
        time.sleep(200)
        starter+=300
    else:
        time.sleep(60)

The price of Nifty-50 will decrease! from 14763.950195312 at 2021-03-30 12:55:10
The price of Nifty-50 will decrease! from 14759.900390625 at 2021-03-30 13:00:33
The price of Nifty-50 will decrease! from 14738.549804688 at 2021-03-30 13:06:01
The price of Nifty-50 will increase! from 14742.599609375 at 2021-03-30 13:10:24
The price of Nifty-50 will increase! from 14746.299804688 at 2021-03-30 13:15:47
The price of Nifty-50 will increase! from 14759.099609375 at 2021-03-30 13:20:09
The price of Nifty-50 will decrease! from 14764.75 at 2021-03-30 13:25:31
The price of Nifty-50 will decrease! from 14792.549804688 at 2021-03-30 13:30:54
The price of Nifty-50 will decrease! from 14782.400390625 at 2021-03-30 13:35:16
The price of Nifty-50 will decrease! from 14795.0 at 2021-03-30 13:40:44
The price of Nifty-50 will decrease! from 14798.549804688 at 2021-03-30 13:45:08


URLError: <urlopen error [Errno -2] Name or service not known>

In [42]:
# starter=timestart
starter=int(time.time()-3600)
# buffPd=pd.DataFrame(columns=['t', 'c', 'o', 'h', 'l', 'v', 'vo', 's'])
Resolution=5
main_df=pd.DataFrame()
while(1):
    url = "https://tvc4.forexpros.com/9e947eed212d70ba03e313a75eb9a1f5/1616696634/56/56/23/history?symbol=17940&resolution="+str(Resolution)+"&from="+str(starter)+"&to="+str(int(time.time()))
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'

    headers={'User-Agent':user_agent,} 

    request=urllib.request.Request(url,None,headers)
    response = urllib.request.urlopen(request)

    data = response.read()
#     print(data)
#     print(len(pd.read_json(data)))
#     print(data)
    try:
        main_df=pd.read_json(data)
#         time.sleep(60)
    except:
        print("No data available yet!")
        time.sleep(60)
    
    if len(main_df)>=13:
        main_df=main_df[(main_df.index < 13)]
        prevPrice=str(np.array(main_df['c'])[-1])
        main_df['diff']=main_df.h-main_df.l
        main_df=main_df.drop(labels=['t','o','h','l','s','vo'],axis=1)
        
        validation_x = preprocess_df(main_df)
        Result=np.argmax(model.predict(np.array(validation_x).reshape(1,12,3)))
        
        dt=datetime.fromtimestamp(time.time()+300).strftime('%Y-%m-%d %H:%M:%S')
        
        if Result==0:
            print("The price of Nifty-50 will decrease! from " +prevPrice+" at "+dt)
        else:
            print("The price of Nifty-50 will increase! from " +prevPrice+" at "+dt)
        time.sleep(200)
        starter+=300
    else:
        time.sleep(60)

The price of Nifty-50 will decrease! from 14848.549804688 at 2021-03-30 14:55:55
The price of Nifty-50 will increase! from 14865.849609375 at 2021-03-30 15:00:17
The price of Nifty-50 will decrease! from 14855.950195312 at 2021-03-30 15:05:40
The price of Nifty-50 will decrease! from 14838.349609375 at 2021-03-30 15:11:03
The price of Nifty-50 will decrease! from 14848.25 at 2021-03-30 15:15:24
The price of Nifty-50 will decrease! from 14841.400390625 at 2021-03-30 15:20:47
The price of Nifty-50 will decrease! from 14844.450195312 at 2021-03-30 15:25:14
The price of Nifty-50 will decrease! from 14834.599609375 at 2021-03-30 15:30:38
The price of Nifty-50 will decrease! from 14845.099609375 at 2021-03-30 15:37:07


KeyboardInterrupt: 